In [2]:
import pandas as pd
pd.set_option("display.max_row", None)
%load_ext autoreload
%autoreload 2

In [3]:
from absbox import API,mkDeal,Endpoints
localAPI = API("http://localhost:8081",check=False)

✅Connected, local lib:0.20.2, server:0.24.3

In [4]:
from absbox import SPV
name = "安逸花 2023 年第六期个人消费贷款资产支持证券"
dates = {"cutoff":"2023-09-19","closing":"2023-12-26","firstPay":"2024-01-26"
        ,"stated":"2028-09-26","poolFreq":"MonthEnd","payFreq":["DayOfMonth",26]}

ast = ["Mortgage"
          ,{"originBalance":169_011_8000,"originRate":["fix",0.238],"originTerm":10
              ,"freq":"Monthly","type":"Level","originDate":"2023-08-01"}
          ,{"currentBalance":150_000_0000 
              ,"currentRate":0.238
              ,"remainTerm":9
              ,"status":"current"}]

pool = {'assets':[ ast ]}

accounts = {"本金分账户":{"balance":0}
            ,"收入分账户":{"balance":0}
            ,"流动性储备":{"balance":15000_000
                         ,"type":{"targetReserve":[("sum",("feeDue","增值税","手续费","服务费")
                                                         ,("bondDueInt","A1","A2"))
                                                   ,2.0]}
                         }}
bonds = {"A1":{"balance":450_000_000
            ,"rate":0.0295
            ,"originBalance":450_000_000
            ,"originRate":0.0295
            ,"startDate":"2023-12-26"
            ,"rateType":{"Fixed":0.0295}
            ,"maturityDate":"2024-10-26"
            ,"bondType":{"PAC":
                           [["2024-01-26",450_000_000]
                           ,["2024-02-26",450_000_000]
                           ,["2024-03-26",450_000_000]
                           ,["2024-04-26",450_000_000]
                           ,["2024-05-26",450_000_000]
                           ,["2024-06-26",450_000_000]
                           ,["2024-07-26",450_000_000]
                           ,["2024-08-26",252_000_000]
                           ,["2024-09-26",162_000_000]
                           ,["2024-10-26",90_000_000]
                           ,["2024-11-20",0]
                           ]}}
        ,"A2":{"balance":660_000_000
            ,"rate":0.03
            ,"originBalance":660_000_000
            ,"originRate":0.03
            ,"startDate":"2023-12-26"
            ,"maturityDate":"2024-12-18"
            ,"rateType":{"Fixed":0.03}
            ,"bondType":{"Sequential":None}}
        ,"B":{"balance":165_000_000
            ,"rate":0.035
            ,"originBalance":165_000_000
            ,"originRate":0.035
            ,"startDate":"2023-12-26"
            ,"maturityDate":"2025-01-26"
            ,"rateType":{"Fixed":0.035}
            ,"bondType":{"Sequential":None}
            }
        ,"R":{"balance":225_000_000
            ,"rate":0.0
            ,"originBalance":225_000_000
            ,"originRate":0.0
            ,"startDate":"2023-12-26"
            ,"maturityDate":"2025-09-26"
            ,"rateType":{"Fixed":0.00}
            ,"bondType":{"Equity":None}
            }       
        }

waterfall = {"default":[
                ["calcFee","增值税","手续费","服务费"]
                ,["calcInt","A1","A2","B"]
                ,["IfElse",["all", [("isPaidOff", "A2","A1","B"),False]
                                 , [("status", "Defaulted"),False]]
                    ,[["transfer","流动性储备","收入分账户",{"reserve":"excess"}]
                     ,["transfer","流动性储备","收入分账户"
                       ,{"formula":("-"
                                     ,("sum",("feeDue","增值税","手续费","服务费")
                                            ,("bondDueInt","A1","A2"))
                                     ,("accountBalance","收入分账户"))}]]
                    ,[["transfer","流动性储备","收入分账户"]]
                 ]
                ,["payFeeBySeq", "收入分账户", ["增值税","手续费"],{'support':["account","本金分账户",['AccountDraw','abcd']]}]
                ,["payFee", "收入分账户", ["服务费"],{"limitCap":50000,'support':["account","本金分账户",['AccountDraw','abcd']]}]
                ,["payInt","收入分账户",["A1","A2"],{'support':["account","本金分账户",['AccountDraw','abcd']]}]
                ,["payInt","收入分账户",["B"],{'support':["account","本金分账户",['AccountDraw','abcd']]}]
                ,["transfer","本金分账户","收入分账户",{"formula":("floorWithZero"
                                                         ,("-"
                                                           ,("reserveGap","流动性储备")
                                                           ,("accountBalance","收入分账户")))}]
                ,["transfer","收入分账户","流动性储备", {"reserve":"gap"}]
                
                ,["IfElse",["status", "Accelerated"]
                          ,[["transfer","收入分账户","本金分账户"]]
                          ,[["transfer","收入分账户","本金分账户",{"冲销":"abcd"}]
                           ,["transfer","收入分账户","本金分账户"]]
                          ]
                # principal distribution
                ,["transfer", "本金分账户", "收入分账户", {"冲销":"abcd"}]
                ,["IfElse", ["status", "Revolving"]
                    ,[["buyAsset",["Current|Defaulted", 1.0, 0.0],"本金分账户"]]
                    ,[["IfElse", ["status", "Accelerated"]
                         ,[["payPrinResidual","本金分账户",["A1","A2"]]]
                         ,[
                             ["If",[("isPaidOff", "A2"),False]
                                  ,['payPrin',"本金分账户",["A1"]]
                                  ,['payPrin',"本金分账户",["A2"]]
                                  ]
                             ,["If",["all"
                                    ,[("isPaidOff", "A1"),True]
                                    ,[("isPaidOff", "A2"),False]]
                                   ,["payPrin","本金分账户",["A2"]]]
                             ,["If",["all"
                                     ,[("isPaidOff", "A1"),False]
                                     ,[("isPaidOff", "A2"),True]]
                                   ,["payPrinResidual","本金分账户",["A1"]]]
                          ]
                      ]
                    ,["payPrin","本金分账户",["B"]]
                    ,["payFee", "本金分账户", ["服务费"]]
                    ,["payPrin", "本金分账户", ["R"]]
                    ,["payIntResidual","本金分账户","R"]]
                 ]
            ],
            
            "endOfCollection":[["bookBy",["formula","abcd"
                                 ,"Credit",("curPoolCollection",None,"Defaults")]]]
            }
collects = [["CollectedInterest","收入分账户"]
            ,["CollectedPrincipal","本金分账户"]
            ,["CollectedPrepayment","本金分账户"]
            ,["CollectedRecoveries","本金分账户"]]

fees = {
    "增值税":{"类型":{"差额费用":[("factor"
                                   ,("资产池累计",None,"利息"),0.0326)
                                   ,("费用支付总额",None,"增值税")]}}
    ,"手续费":{"type":{"fixFee":100}}
    ,"服务费":{"type":{"fixFee":100}}
}         

triggers = {"BeforeCollect": # optional
              {"加速清偿":
                {"condition":[("cumPoolDefaultedRate",),">"
                              ,[["2024-12-26",0.06]
                               ,["2025-12-26",0.08]]]
                ,"effects":("newStatus","Accelerated")
                ,"status":False
                ,"curable":False}
              ,
              "GoToAmort":
                {"condition":[">=","2024-07-31"]
                ,"effects":("newStatus","Amortizing")
                ,"status":False
                ,"curable":False}
              }            
}

ledgers = {"abcd":{"balance":0.0}}
         
deal_data = {
    "name":name
    ,"dates":dates
    ,"pool":pool
    ,"accounts":accounts
    ,"bonds":bonds
    ,"fees":fees
    ,"waterfall":waterfall
    ,"collect":collects
    ,"triggers":triggers
    ,"ledgers":ledgers
    ,"status":("PreClosing","Revolving")
}

from absbox import mkDeal
AYN_AUTO_2021_2 = mkDeal(deal_data)  ## now a generic class created

In [40]:
revol_asset = ["Mortgage"
                ,{"originBalance":2200,"originRate":["fix",0.22],"originTerm":12
                  ,"freq":"Monthly","type":"Level","originDate":"2023-07-01"}
                  ,{"currentBalance":2200
                  ,"currentRate":0.22
                  ,"remainTerm":12
                  ,"status":"current"}]

r = localAPI.run(AYN_AUTO_2021_2
                 ,poolAssump = ("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                          ,None
                                          ,None)
                 ,runAssump = [("revolving"
                                ,["constant",revol_asset]
                                ,("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                          ,None
                                          ,None))
                              ,("pricing"
                              ,{"date":"2023-12-26"
                                ,"curve":[["2021-01-01",0.025]
                                         ,["2024-08-01",0.025]]})
                              ]
                 ,read=True)

⠋0m


In [34]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2023-12-26,1.026577e+09,4.700568e+08,79904039.15,0,3366455.28,0,3366455.28,0.238000,None,None,4.700568e+08,0,0,3366455.28,0,3366455.28
2023-12-31,1.026577e+09,0.000000e+00,0.00,0,0.00,0,0.00,0.238000,None,None,4.700568e+08,0,0,3366455.28,0,3366455.28
2024-01-31,1.411260e+09,1.626682e+08,20342725.32,0,875901.95,0,875901.95,0.228507,None,None,6.327249e+08,0,0,4242357.23,0,4242357.23
2024-02-29,1.386010e+09,2.069779e+08,27144007.86,0,1204124.08,0,1204124.08,0.223914,None,None,7.984778e+08,0,0,4978719.13,0,4978719.13
2024-03-31,1.390445e+09,2.246093e+08,26433427.96,0,1106315.02,0,1106315.02,0.221756,None,None,9.673830e+08,0,0,5534700.93,0,5534700.93
2024-04-30,1.390048e+09,2.460916e+08,26259229.00,0,1186363.83,0,1186363.83,0.220756,None,None,1.139491e+09,0,0,5984421.20,0,5984421.20
2024-05-31,1.387983e+09,2.692195e+08,25994063.89,0,1147782.33,0,1147782.33,0.220303,None,None,1.314868e+09,0,0,6277156.88,0,6277156.88
2024-06-30,1.383506e+09,2.943595e+08,25692232.13,0,1184262.99,0,1184262.99,0.220104,None,None,1.493570e+09,0,0,6429760.55,0,6429760.55
2024-07-31,1.558816e+09,1.395682e+08,25342866.09,0,1142379.86,0,1142379.86,0.220103,None,None,1.493570e+09,0,0,6429760.55,0,6429760.55


In [31]:
r['ledgers']['abcd']

,balance,amount,comment
date,,,
2023-12-26,3366455.28,3366455.28,<TxnDirection:Credit>
2023-12-31,3366455.28,0.00,<TxnDirection:Credit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,0.00,-3366455.28,<TxnDirection:Credit>
2024-01-26,0.00,0.00,<TxnDirection:Credit>
2024-01-31,875901.95,875901.95,<TxnDirection:Credit>


In [55]:
r['ledgers']['abcd']

,balance,amount,comment
date,,,
2023-12-26,3366455.28,3366455.28,<TxnDirection:Credit>
2023-12-31,3366455.28,0.00,<TxnDirection:Credit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,3366455.28,0.00,<TxnDirection:Debit>
2024-01-26,0.00,-3366455.28,<TxnDirection:Credit>
2024-01-26,0.00,0.00,<TxnDirection:Credit>
2024-01-31,875901.95,875901.95,<TxnDirection:Credit>


In [67]:
r['accounts']['收入分账户']

,balance,change,memo
date,,,
2023-12-26,52844353.71,52844353.71,<Pool:CollectedInterest>
2023-12-31,52844353.71,0.00,<Pool:CollectedInterest>
2024-01-26,58780282.69,5935928.98,"<TransferBy:流动性储备,收入分账户,TillSource>"
2024-01-26,58780282.69,0.00,"<TransferBy:流动性储备,收入分账户,DS (Substract [Sum [Cu..."
2024-01-26,57057456.76,-1722825.93,<SeqPayFee:增值税手续费>
2024-01-26,57057356.76,-100.00,<SeqPayFee:服务费>
2024-01-26,54248247.18,-2809109.58,<PayInt:A1A2>
2024-01-26,53757767.73,-490479.45,<PayInt:B>
2024-01-26,53757767.73,0.00,"<TransferBy:本金分账户,收入分账户,DS (FloorWithZero (Sub..."


In [32]:
r['accounts']['本金分账户']

,balance,change,memo
date,,,
2023-12-26,0.000000e+00,0.000000e+00,<Pool:CollectedRecoveries>
2023-12-26,0.000000e+00,0.000000e+00,<Pool:CollectedPrepayment>
2023-12-26,4.700568e+08,4.700568e+08,<Pool:CollectedPrincipal>
2023-12-31,4.700568e+08,0.000000e+00,<Pool:CollectedRecoveries>
2023-12-31,4.700568e+08,0.000000e+00,<Pool:CollectedPrepayment>
2023-12-31,4.700568e+08,0.000000e+00,<Pool:CollectedPrincipal>
2024-01-26,4.700568e+08,0.000000e+00,<SupportDraw:>
2024-01-26,4.700568e+08,0.000000e+00,<SupportDraw:>
2024-01-26,4.700568e+08,0.000000e+00,<SupportDraw:>


In [50]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-01-26,450000000,1127465.75,0,0.0295,1.127466e+06,<PayInt:A1>
2024-02-26,450000000,1127465.75,0,0.0295,1.127466e+06,<PayInt:A1>
2024-03-26,450000000,1054726.02,0,0.0295,1.054726e+06,<PayInt:A1>
2024-04-26,450000000,1127465.75,0,0.0295,1.127466e+06,<PayInt:A1>
2024-05-26,450000000,1091095.89,0,0.0295,1.091096e+06,<PayInt:A1>
2024-06-26,450000000,1127465.75,0,0.0295,1.127466e+06,<PayInt:A1>
2024-07-26,450000000,1091095.89,0,0.0295,1.091096e+06,<PayInt:A1>
2024-08-26,252000000,1127465.75,198000000,0.0295,1.991275e+08,"[<PayInt:A1>, <PayPrin:A1>]"
2024-09-26,162000000,631380.82,90000000,0.0295,9.063138e+07,"[<PayInt:A1>, <PayPrin:A1>]"


In [58]:
r['bonds']['A2']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-01-26,6.600000e+08,1681643.83,0.000000e+00,0.03,1.681644e+06,<PayInt:A2>
2024-02-26,6.600000e+08,1681643.83,0.000000e+00,0.03,1.681644e+06,<PayInt:A2>
2024-03-26,6.600000e+08,1573150.68,0.000000e+00,0.03,1.573151e+06,<PayInt:A2>
2024-04-26,6.600000e+08,1681643.83,0.000000e+00,0.03,1.681644e+06,<PayInt:A2>
2024-05-26,6.600000e+08,1627397.26,0.000000e+00,0.03,1.627397e+06,<PayInt:A2>
2024-06-26,6.600000e+08,1681643.83,0.000000e+00,0.03,1.681644e+06,<PayInt:A2>
2024-07-26,6.600000e+08,1627397.26,0.000000e+00,0.03,1.627397e+06,<PayInt:A2>
2024-08-26,5.994169e+08,1681643.83,6.058314e+07,0.03,6.226478e+07,"[<PayInt:A2>, <PayPrin:A2>]"
2024-09-26,5.381734e+08,1527281.32,6.124348e+07,0.03,6.277076e+07,"[<PayInt:A2>, <PayPrin:A2>]"


In [62]:
r['bonds']['B']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-01-26,1.650000e+08,490479.45,0.000000e+00,0.035,4.904795e+05,<PayInt:B>
2024-02-26,1.650000e+08,490479.45,0.000000e+00,0.035,4.904795e+05,<PayInt:B>
2024-03-26,1.650000e+08,458835.61,0.000000e+00,0.035,4.588356e+05,<PayInt:B>
2024-04-26,1.650000e+08,490479.45,0.000000e+00,0.035,4.904795e+05,<PayInt:B>
2024-05-26,1.650000e+08,474657.53,0.000000e+00,0.035,4.746575e+05,<PayInt:B>
2024-06-26,1.650000e+08,490479.45,0.000000e+00,0.035,4.904795e+05,<PayInt:B>
2024-07-26,1.650000e+08,474657.53,0.000000e+00,0.035,4.746575e+05,<PayInt:B>
2024-08-26,1.650000e+08,490479.45,0.000000e+00,0.035,4.904795e+05,"[<PayInt:B>, <PayPrin:B>]"
2024-09-26,1.650000e+08,490479.45,0.000000e+00,0.035,4.904795e+05,"[<PayInt:B>, <PayPrin:B>]"


In [63]:
r['bonds']['R']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-08-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2024-09-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2024-10-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2024-11-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2024-12-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2025-01-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2025-02-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2025-03-26,2.250000e+08,0,0.00,0,0.00,"[<PayPrin:R>, <PayYield:R>]"
2025-04-26,1.794890e+08,0,45510979.01,0,45510979.01,"[<PayPrin:R>, <PayYield:R>]"


In [41]:
r['pricing']

,pricing,face,WAL,duration,convexity,accure interest
A1,4.516386e+08,100.364134,0.76,0.761861,1.29,0
A2,6.632992e+08,100.499875,0.95,0.946253,1.77,0
B,1.669481e+08,101.180658,1.16,1.144510,2.35,0
R,5.608880e+08,249.283543,1.25,1.380417,3.14,0
